In [ ]:
"""
Downloading script for soccer logs public open dataset:
https://figshare.com/collections/Soccer_match_event_dataset/4415000/2
Please cite the source as: - Pappalardo et al., (2019) A public data set of spatio-temporal match events in soccer competitions, Nature Scientific Data 6:236, https://www.nature.com/articles/s41597-019-0247-7
"""

import requests, zipfile, json, io


dataset_links = {

'matches' : 'https://ndownloader.figshare.com/files/14464622',
'events' : 'https://ndownloader.figshare.com/files/14464685',
'players' : 'https://ndownloader.figshare.com/files/15073721',
'teams': 'https://ndownloader.figshare.com/files/15073697',
}


r = requests.get(dataset_links['matches'], stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("data/matches")

r = requests.get(dataset_links['events'], stream=True)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("data/events")
#
r = requests.get(dataset_links['teams'], stream=False)
print (r.text, file=open('data/teams.json','w'))


r = requests.get(dataset_links['players'], stream=False)
print (r.text, file=open('data/players.json','w'))

#Corrupted file, you will need to adjust the end manually to read it.
r = requests.get(dataset_links['referees'], stream=False)
print (r.text, file=open('data/referees.json','w'))

In [1]:
"""Run next two cells for all leagues extraction except France"""
import requests, zipfile, json, io
from pandas.io.json import json_normalize
import numpy as np
import re
import pandas as pd
file_name = ["\events_England.json", "\events_Germany.json", "\events_Italy.json", "\events_Spain.json", "\events_France.json"]
directory = #Wherever you placed these files
all_events = pd.DataFrame()

for i in file_name:
    with open (directory + i) as f:
        data = json.load(f)
    df = json_normalize(data, sep = '_')
    if all_events.empty:
        all_events = df
    else:
        all_events = pd.concat([all_events, df], join = 'outer', sort = False)


In [2]:
#Without France
file_name_matches = ["\matches_England.json", "\matches_Germany.json", "\matches_Italy.json", "\matches_Spain.json", "\matches_France.json"]
directory_matches = #Wherever you placed these files
all_matches = pd.DataFrame()

for i in file_name_matches:
    with open (directory_matches + i) as f:
        data = json.load(f)
    df = json_normalize(data, sep = '_')
    if all_matches.empty:
        all_matches = df
    else:
        all_matches = pd.concat([all_matches, df], join = 'outer', sort = False)


In [3]:
with open("Wherever you placed this file") as f:
    teams = json.load(f)
    df_teams = json_normalize(teams, sep =",")
with open("Wherever you placed this file") as f:
    players = json.load(f)
    df_players = json_normalize(players, sep =",")
with open("Wherever you placed this file") as f:
    refe = json.load(f)
    referees= json_normalize(refe, sep =",")

In [ ]:
#construct columns of home and away team
def match_home_away_name(x):
    home = []
    away = []
    ref_Id = []
    for element in x.label:
        temp = re.split("[-,]", element)
        if temp[0] == "Saint":
            home.append(temp[0] +"-" + temp[1])
            away.append(temp[2])
        else:
            home.append(temp[0])
            away.append(temp[1])
    for element in x.referees:
        if element:
            ref_Id.append(element[0]["refereeId"])
        else:
            ref_Id.append(0)
    x["home"] = home
    x["away"] = away
    x["refId"] = ref_Id
    return x

In [ ]:
#merge
def merging(df, df_matches, df_teams, df_players, df_referees):
    df_referees = df_referees.rename(columns = {"wyId": "refId_ref", "shortName" : "shortNameRef"})
    df_players = df_players.rename(columns = {"shortName": "shortNamePlayers"})
    merged_team = pd.merge(df, df_matches[["home", "away", "wyId", "refId", "gameweek", "competitionId"]], how='left', left_on=["matchId"], right_on = ["wyId"]).drop(["wyId"], axis = 1)
    merged_team = pd.merge(merged_team, df_referees[["refId_ref", "shortNameRef"]], how='left', left_on=["refId"], right_on = ["refId_ref"]).drop(["refId_ref"], axis = 1)
    merged_team2 = pd.merge(merged_team, df_teams[["name", "wyId"]], how='left', left_on=["teamId"], right_on = ["wyId"]).drop(["wyId"], axis = 1)
    merged = pd.merge(merged_team2,df_players[["shortNamePlayers","wyId", "role,name"]], how='left', left_on=["playerId"], right_on = ["wyId"]).drop(["wyId"], axis = 1)
    return merged

In [ ]:
#We need adjustments for France DB idiosyncracies given the way we will handle the data
def france_adj(merged):
    var = "Saint-" +"\\"+ "u00c9tienne"
    temp = merged.name.replace({"Angers": "Angers SCO"})
    temp1 = merged.away.replace({" Saint": var})
    merged.name = temp
    merged.away = temp1
    return merged

In [ ]:
def score(merged):
    home_score = np.zeros(len(merged), int)
    away_score = np.zeros(len(merged), int)
    temp = merged.matchId[0]
    counter_home = 0
    counter_away= 0
    minute_counter = 0
    for idx, val in enumerate(merged.tags):
        #debugging the loop for empty arrays
        if val:
            #resetting values if new game
            if(temp != merged.matchId[idx]):
                counter_home = 0
                counter_away = 0
                minute_counter = 0
                temp = merged.matchId[idx]
                #in case first event is a goal or own goal
                if({'id': 101} == val[0]):
                    #to avoid for double counting event (like a shot that was stopped but went in will have the goal event twice)
                    if(abs(merged.eventSec[idx] - minute_counter) < 10):
                        home_score[idx] = counter_home
                        away_score[idx] = counter_away
                        pass
                    else:
                        #check which team scored
                        if (merged.name[idx].split() == merged.home[idx].split()):
                            counter_home += 1
                            home_score[idx] = counter_home
                            away_score[idx] = counter_away
                            minute_counter = merged.eventSec[idx]
                        elif (merged.name[idx].split() == merged.away[idx].split()):
                            counter_away += 1
                            away_score[idx] = counter_away
                            home_score[idx] = counter_home
                            minute_counter = merged.eventSec[idx]
                        #debug
                        else:
                            print(merged.name[idx], merged.away[idx], merged.home[idx], idx, val)
                elif({'id': 102} == val[0]):
                    if(abs(merged.eventSec[idx] - minute_counter) < 10):
                        home_score[idx] = counter_home
                        away_score[idx] = counter_away
                        pass
                    else:
                        if (merged.name[idx].split() == merged.home[idx].split()):
                            counter_away += 1
                            away_score[idx] = counter_away
                            home_score[idx] = counter_home
                            minute_counter = merged.eventSec[idx]
                        elif (merged.name[idx].split() == merged.away[idx].split()):
                            counter_home += 1
                            home_score[idx] = counter_home
                            away_score[idx] = counter_away
                            minute_counter = merged.eventSec[idx]
                        else:
                            print(merged.name[idx], merged.away[idx], merged.home[idx], idx, val)
            elif(temp == merged.matchId[idx]):
                if({'id': 101} == val[0]):
                    if(abs(merged.eventSec[idx] - minute_counter) < 10):
                        home_score[idx] = counter_home
                        away_score[idx] = counter_away
                        pass
                    else:
                    #check which team scored
                        if (merged.name[idx].split() == merged.home[idx].split()):
                            counter_home += 1
                            home_score[idx] = counter_home
                            away_score[idx] = counter_away
                            minute_counter = merged.eventSec[idx]
                        elif (merged.name[idx].split() == merged.away[idx].split()):
                            counter_away += 1
                            away_score[idx] = counter_away
                            home_score[idx] = counter_home
                            minute_counter = merged.eventSec[idx]
                        #debug
                        else:
                            print(merged.name[idx], merged.away[idx], merged.home[idx], idx, val)
                elif({'id': 102} == val[0]):
                    if(abs(merged.eventSec[idx] - minute_counter) < 10):
                        home_score[idx] = counter_home
                        away_score[idx] = counter_away
                        pass
                    else:
                        if (merged.name[idx].split() == merged.home[idx].split()):
                            counter_away += 1
                            away_score[idx] = counter_away
                            home_score[idx]= counter_home
                            minute_counter = merged.eventSec[idx]

                        elif (merged.name[idx].split() == merged.away[idx].split()):
                            counter_home += 1
                            home_score[idx] = counter_home
                            away_score[idx] = counter_away
                            minute_counter = merged.eventSec[idx]
                        else:
                            print(merged.name[idx], merged.away[idx], merged.home[idx], idx, val) 
                else:
                    home_score[idx] = counter_home
                    away_score[idx] = counter_away
        else:
            home_score[idx] = counter_home
            away_score[idx] = counter_away
    merged["homeScore"] = home_score
    merged["awayScore"] = away_score
    return merged

In [ ]:
#transform seconds to minutes

"""From HERE START WORKING ON FOUL ARRAY ONLY"""
def minute_columns(df):
    df = df[df.eventId == 2]
    period = np.array(df.matchPeriod)
    adjusted_sec = np.array(df.eventSec)
    for i in range(len(period)):
        if period[i] == "1H":
            adjusted_sec[i] /= 60
        elif period[i] == "2H":
            adjusted_sec[i] = (adjusted_sec[i] / 60) + 45
    adjusted_sec = (np.round(adjusted_sec,2))
    df["eventMinute"] = adjusted_sec
    return df

In [ ]:
#Create x & y (coordinates) columns
def x_y_creation(merged):
    x_array = np.zeros(len(merged))
    y_array = np.zeros(len(merged))
    for idx, elements in enumerate(merged.positions):
        x_array[idx] = elements[0]["x"]
        y_array[idx] = elements[0]["y"]
    merged["x"] = x_array
    merged["y"] = y_array
    return merged

In [ ]:
#handling of tags to create name values instead of IDs
def boolies(df_foul):
    boolies = list(np.zeros(len(df_foul),int))
    for idx, val in enumerate(df_foul.tags):
        if([{'id': 1701}] == val):
            boolies[idx] = "Red Card"
        elif([{'id': 1702}] == val):
            boolies[idx] = "Yellow Card"
        elif([{'id': 1703}] == val):   
            boolies[idx] = "Second Yellow"
    df_foul["card"] = boolies
    return df_foul

In [ ]:
def booking_faults(final):
    cum_faults_home = np.zeros(len(final))
    cum_faults_away = np.zeros(len(final))
    cum_cards_home = np.zeros(len(final))
    cum_cards_away = np.zeros(len(final))
    cum_faults = np.zeros(len(final))
    cum_cards = np.zeros(len(final))
    for row in range(1,len(final)):
        if final['matchId'].iloc[row] == final['matchId'].iloc[row-1]:
            cum_faults[row] = cum_faults[row-1]+1
            if final['name'].iloc[row].split() == final['home'].iloc[row].split():
                cum_faults_home[row] = cum_faults_home[row-1] + 1
                cum_faults_away[row] = cum_faults_away[row-1]  
                if final["card"].iloc[row] != 0:
                    cum_cards[row] = cum_cards[row-1] +1
                    cum_cards_home[row] = cum_cards_home[row-1]+1
                    cum_cards_away[row] = cum_cards_away[row-1]
                else:
                    cum_cards[row] = cum_cards[row-1]
                    cum_cards_home[row] = cum_cards_home[row-1]
                    cum_cards_away[row] = cum_cards_away[row-1]   
            else:
                cum_faults_home[row] = cum_faults_home[row-1]
                cum_faults_away[row] = cum_faults_away[row-1] + 1 
                if final["card"].iloc[row] != 0:
                    cum_cards[row] = cum_cards[row-1] +1
                    cum_cards_home[row] = cum_cards_home[row-1]
                    cum_cards_away[row] = cum_cards_away[row-1]+1  
                else:
                    cum_cards[row] = cum_cards[row-1]
                    cum_cards_home[row] = cum_cards_home[row-1]
                    cum_cards_away[row] = cum_cards_away[row-1]   
        else:
            cum_faults_home[row] = 0
            cum_faults_away[row] = 0
            cum_cards_home[row] = 0
            cum_cards_away[row] = 0
            cum_faults[row] = 0
            cum_cards[row] = 0
    print(cum_faults_home[3])
    final["cum_faults_diff"] = np.subtract(cum_faults_home, cum_faults_away)
    final["cum_cards_diff"] = np.subtract(cum_cards_home, cum_cards_away)
    final["cum_cards"] = cum_cards
    final["cum_faults"] = cum_faults
    return final

In [ ]:
#create temp dic hosting pairs of keys(matchId) and values(PlayerId) for players that received a yellow card
def card_per_player(merged):
    temp_dic = dict()
    array_tfill = np.zeros(len(merged.card))
    temp = merged.matchId.iloc[0]
    for idx, val in enumerate(merged.card):
        #resetting values if new game
        if(temp != merged.matchId.iloc[idx]):
            temp = merged.matchId.iloc[idx]
            #in case a yellow card was attributed
            if(merged.card.iloc[idx] == "Yellow Card"):
                #check if current key / values(array) exists
                if temp in temp_dic:
                    temp_dic[temp].append(merged.playerId.iloc[idx])
                else:
                    temp_dic[temp] = [merged.playerId.iloc[idx]]
            #else check if player had a card and print one in array_tfill
            else:
                #first see if game was referenced
                if temp in temp_dic:
                    #then see if playerId was referenced
                    if merged.playerId.iloc[idx] in temp_dic[temp]:
                        array_tfill[idx] = 1
                else:
                    pass
        else:
            #in case a yellow card was attributed
            if(merged.card.iloc[idx] == "Yellow Card"):
                #check if current key / values(array) exists
                if temp in temp_dic:
                    temp_dic[temp].append(merged.playerId.iloc[idx])
                else:
                    temp_dic[temp] = [merged.playerId.iloc[idx]]
            #else check if player had a card and print one in array_tfill
            else:
                #first see if game was referenced
                if temp in temp_dic:
                    #then see if playerId was referenced
                    if merged.playerId.iloc[idx] in temp_dic[temp]:
                        array_tfill[idx] = 1
                else:
                    pass
    merged["Has a Yellow Card"] = array_tfill
    return merged

In [ ]:
#obv this is different than a MA
def faultmovingavg(final):
    temp_arr = list()
    array_tfill = np.zeros(len(final.eventMinute))
    temp = final.matchId.iloc[0]
    #loop through the df
    for idx, val in enumerate(final.eventMinute):
        #pass first loop for indexing purposes
        if idx == 0:
            temp_arr.append(final.eventMinute.iloc[idx])
        else:
            #check if not same game
            if(temp != final.matchId.iloc[idx]): 
                temp = final.matchId.iloc[idx]
                temp_arr.clear()
                temp_arr.append(final.eventMinute.iloc[idx])
            else:
                for element in temp_arr:
                    if (final.eventMinute.iloc[idx] - element > 10):
                        temp_arr.remove(element)
                    else:
                        pass
                temp_arr.append(final.eventMinute.iloc[idx])
                
                #preventing division by 0 when argmax = argmin
                argMax = np.argmax(temp_arr)
                if (len(temp_arr) > 1):
                    array_tfill[idx] = float(len(temp_arr))/ float(10)
                else:
                    #in case match changed and just the first foul happened after the first 10 mn : divide by time elapsed
                    if(temp != final.matchId.iloc[idx-1]):
                        array_tfill[idx] = float(len(temp_arr))/ float((temp_arr[argMax]))
                    #if same game but lot of time passed between fouls : divide by time since last foul
                    else:
                        array_tfill[idx] = float(len(temp_arr))/ float((temp_arr[argMax] - final.eventMinute.iloc[(idx-1)]))
    final["FaultMovingAvg"] = array_tfill
    return final

In [ ]:
def cumperplayer(final):
    temp_dic = dict()
    array_tfill = np.zeros(len(final.eventMinute))
    temp = final.matchId.iloc[0]
    #loop through the df
    for idx, val in enumerate(final.playerId):
        #pass first loop for indexing purposes
        if idx == 0:
            temp_dic[val] = 1
            array_tfill[idx] = temp_dic[val]
        else:
            #check if not same game
            if(temp != final.matchId.iloc[idx]): 
                temp = final.matchId.iloc[idx]
                temp_dic.clear()
                temp_dic[val] = 1
                array_tfill[idx] = temp_dic[val]
            else:
                if val in temp_dic:
                    temp_dic[val] +=1
                    array_tfill[idx] = temp_dic[val]
                else:
                    temp_dic[val] =1 
                    array_tfill[idx] = temp_dic[val]
                
    final["CumFaultOfPlayerCommittingFault"] = array_tfill
    return final

In [ ]:
#Creation of adjusted DB for our purpose (model fitting, etc)
def dbcreation_noref(all_games):
    df = pd.DataFrame()
    
    #Coordinates
    df["x"] = all_games["x"]
    df["y"] = all_games["y"]
    
    #Minutes
    df["minute"] = all_games["eventMinute"]
    
    #score_diff
    score_diff = all_games["homeScore"] - all_games["awayScore"]
    df["score_diff"] = score_diff
    
    #append cum fouls,card and diffs, fault mvg avg for past 10 mns, cumfaultofsameplayer, has a yellow card
    df["cum_faults_diff"] = all_games["cum_faults_diff"]
    df["cum_cards_diff"] = all_games["cum_cards_diff"]
    df["cum_cards"] = all_games["cum_cards"]
    df["cum_faults"] = all_games["cum_faults"]
    df["fault_moving_avg"] = all_games["FaultMovingAvg"]
    df["cum_fault_of_player"] = all_games["CumFaultOfPlayerCommittingFault"]
    df["Has_a_yellow_card"] = all_games["Has a Yellow Card"]
    
    #nesting IDs
    df["playerId"] = all_games["playerId"]
    df["teamId"] = all_games["teamId"]
    df["competitionId"] = all_games["competitionId"]
    df["refId"] = all_games["refId"]
    df["gameweek"] = all_games["gameweek"]
    
    #type of foul
    dum_foul = pd.get_dummies(all_games.subEventName)
    #Drop a column to avoid the colinearity issue of the dummy variable trap
    df = pd.concat([df, dum_foul], axis = "columns").drop(["Foul","Simulation"], axis = "columns")
    
    #Position of Player
    dum_position = pd.get_dummies(all_games["role,name"])
    df = pd.concat([df, dum_position], axis = "columns").drop(["Goalkeeper"], axis = "columns")

    
    #Y
    dum = pd.get_dummies(all_games.card)
    Y = dum[["Red Card","Second Yellow", "Yellow Card"]].sum(axis = 1)
    df["card_y"] = Y
    
    return df

In [15]:
matches = match_home_away_name(all_matches)


In [17]:
merged = merging(all_events,all_matches,df_teams,df_players,referees)


In [20]:
final = france_adj(merged)

In [ ]:
final1 = score(final)

In [ ]:
final2 = minute_columns(final1)

In [ ]:
final3 = x_y_creation(final2)

In [ ]:
final4 = boolies(final3)

In [ ]:
final5 = booking_faults(final4)

In [ ]:
final6 = card_per_player(final5)

In [ ]:
final7 = faultmovingavg(final6)

In [ ]:
final8 = cumperplayer(final7)

In [5]:
df_final = dbcreation_noref(final8)